This Jupyter notebook will focus on working with Dash Callbacks  
We will : EXTRACT THE AVERAGE MONTHLY DELAY TIME AND SEE HOW IT CHANGES OVER THE YEARS FROM 2010 to 2020  
Tasks to do will be:  
TASK 1:  
Import the required libraries and read the dataset  
TASK 2:  
Create an application layout and add title to dashboard using H1 componenet  
TASK 3:  
Add input text box using core input and output components  
Add the callback decorator and generate the line chart within the callback function

In [41]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

In [42]:
# Creating the dash application layout
app = dash.Dash(__name__)

app.layout =html.Div(
        children=[
            html.H1('Airline Performance Dashboard', style={'text-align':'center', 'color': 'pink','font-size':'40'}),
            html.Div(['Input Year',dcc.Input(id='input-year', value='2010', 
                                type='number', style={'height':'50px', 'font-size': 35}),],
                    style={'font-size': 40}),
                    html.Br(),
                    html.Br(),
                    html.Div(dcc.Graph(id='line-plot'))
            
        ]
    )

#Now add the application callback function 
# In python @app.callback is a decorator used in Dash Frameworks to specify that a function should be called when an input component changes its value.
# The Input and Output functions are used to define the inputs and outputs of a callback function
# The core idea of this application is to get year as user input(input) and update the dashboard(output) in real time with the help of a callback function
# Steps:
# Define the callback decorator
# Define the callback function that uses the user input provided to perform computation
# Create graph and return it as an output



# add callback decorator
@app.callback( Output(component_id='line-plot', component_property='figure'),
               Input(component_id='input-year', component_property='value'))
# Add computation to callback function and return graph
def get_graph(entered_year):
    # Select 2019 data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    
    # Group the data by Month and compute average over arrival delay time.
    line_data = df.groupby('Month')['ArrDelay'].mean().reset_index()
    fig = go.Figure(data=go.Scatter(x=line_data['Month'], y=line_data['ArrDelay'], mode='lines', marker=dict(color='green')))
    fig.update_layout(title='Month vs Average Flight Delay Time', xaxis_title='Month', yaxis_title='ArrDelay')
    return fig




# Run the app
if __name__ == '__main__':
    app.run_server(port=80)

##### PRACTICE QUESTION   
Lets use bar chart to extract the number of flights from a specific airline which goes to a destination

In [43]:
app = dash.Dash(__name__)
app.layout = html.Div( children=[
    html.H1('Total number of flights to the destination state split by reporting air',style={'text-align':'center', 'color':'pink', 'font-size':50}),
    html.Div(['Input Year', dcc.Input(id='input-year',value='2010',type='number', style={'height':'50px', 'font-size': 35}),],
             style={'font-size': 40}),
    html.Br(),
    html.Br(),
    html.Div(dcc.Graph(id='bar-plot'))
]           
)

# now defining callback graph function
# add callback decorator
@app.callback( Output(component_id='bar-plot', component_property='figure'),
               Input(component_id='input-year', component_property='value'))
# Add computation to callback function and return graph
def get_graph(entered_year):
    df =  airline_data[airline_data['Year']==int(entered_year)]
    bar_data = df.groupby('DestState')['Flights'].sum().reset_index()
    fig = px.bar(bar_data, x= "DestState", y= "Flights", title='Total number of flights to the destination state split by reporting airline') 
    fig.update_layout(title='Flights to Destination State', xaxis_title='DestState', yaxis_title='Flights')
    return fig     
# Run the app
if __name__ == '__main__':
    app.run_server(port=80)